In [ ]:
#RNN(Recurrent Neural Network)
>best suitable for sequence data
    text data:
        >food was good
        >i will let you know
        >happy birthday
        etc.
    number sequence
        >1,2,3,4,5,6,7,?
        >10,20,30,40,?
        etc.
    timeseries data
        >2020----->50k
        >2021----->54k
        >2022----->60
        .
        .
        2025------>120000
        2027------>?

In [31]:
#sample=document=str
doc1='fo@od was Good'
doc2='taste was not good'
doc3='will not re$commend to others'
doc4='service was very prompt'
doc5='awesome Food'
import numpy as np

target=np.array([1,0,0,1,1])

#corpus=list or 1d array of docs
corpus=[doc1,doc2,doc3,doc4,doc5]
corpus2=list(map(str.lower,corpus))
print(corpus2)

import re
def clean_punch(doc):
    doc=re.sub('[^a-z ]','',doc)
    return doc

corpus3=list(map(clean_punch,corpus2))
print(corpus3)

['fo@od was good', 'taste was not good', 'will not re$commend to others', 'service was very prompt', 'awesome food']
['food was good', 'taste was not good', 'will not recommend to others', 'service was very prompt', 'awesome food']


In [2]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [33]:
tk=Tokenizer()
tk.fit_on_texts(corpus3)

In [34]:
tk.index_word

{1: 'was',
 2: 'food',
 3: 'good',
 4: 'not',
 5: 'taste',
 6: 'will',
 7: 'recommend',
 8: 'to',
 9: 'others',
 10: 'service',
 11: 'very',
 12: 'prompt',
 13: 'awesome'}

In [35]:
seqs=tk.texts_to_sequences(corpus3)
print(seqs)

[[2, 1, 3], [5, 1, 4, 3], [6, 4, 7, 8, 9], [10, 1, 11, 12], [13, 2]]


In [36]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [37]:
pad_sequences(seqs,padding='post')

array([[ 2,  1,  3,  0,  0],
       [ 5,  1,  4,  3,  0],
       [ 6,  4,  7,  8,  9],
       [10,  1, 11, 12,  0],
       [13,  2,  0,  0,  0]], dtype=int32)

In [38]:
pad_sequences(seqs,padding='pre')

array([[ 0,  0,  2,  1,  3],
       [ 0,  5,  1,  4,  3],
       [ 6,  4,  7,  8,  9],
       [ 0, 10,  1, 11, 12],
       [ 0,  0,  0, 13,  2]], dtype=int32)

In [39]:
pad_sequences(seqs)

array([[ 0,  0,  2,  1,  3],
       [ 0,  5,  1,  4,  3],
       [ 6,  4,  7,  8,  9],
       [ 0, 10,  1, 11, 12],
       [ 0,  0,  0, 13,  2]], dtype=int32)

In [40]:
pad_sequences(seqs,padding='pre',maxlen=4,truncating='post')

array([[ 0,  2,  1,  3],
       [ 5,  1,  4,  3],
       [ 6,  4,  7,  8],
       [10,  1, 11, 12],
       [ 0,  0, 13,  2]], dtype=int32)

In [41]:
pad_seqs=pad_sequences(seqs,padding='pre',maxlen=4,truncating='post')

In [42]:
from tensorflow.keras.utils import to_categorical

In [43]:
X=to_categorical(pad_seqs)
X.shape

(5, 4, 14)

In [44]:
target

array([1, 0, 0, 1, 1])

In [45]:
from tensorflow.keras.layers import SimpleRNN,Dense
from tensorflow.keras.models import Sequential

In [46]:
model=Sequential()
model.add(SimpleRNN(units=10,activation='tanh'))
model.add(Dense(units=5,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,target,batch_size=2,epochs=5)

Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.6000 - loss: 0.6608
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6000 - loss: 0.6461 
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6000 - loss: 0.6319
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6000 - loss: 0.6195
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6000 - loss: 0.6068


In [47]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import SimpleRNN,Dense
from tensorflow.keras.models import Sequential

In [72]:
df=pd.read_csv('f:/ML_Dataset/food_review_dataset.csv')
df

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
1995,Everything was on point — from salads to ice c...,1
1996,Highly recommend trying the burger. Simply dis...,1
1997,Poor service and friendly hygiene standards.,0
1998,This is my go-to place for soup. Never disappo...,1


In [73]:
df['length']=list(map(len,df.Review))
df.length.value_counts()

length
55     108
53     107
51      61
60      56
54      55
      ... 
129      1
108      1
145      1
136      1
149      1
Name: count, Length: 134, dtype: int64

In [74]:
corpus=df.Review
y=df.Liked


In [75]:
corpus2=list(map(str.lower,corpus))

In [89]:
def clean_punch(doc):
    doc=re.sub('[^a-z ]','',doc)
    return doc

corpus3=list(map(clean_punch,corpus2))
tk=Tokenizer()
tk.fit_on_texts(corpus3)
print(len(tk.index_word))
seqs=tk.texts_to_sequences(corpus3)
pad_seqs=pad_sequences(seqs,padding='pre',maxlen=70,truncating='post')
X=to_categorical(pad_seqs)
X.shape

2064


(2000, 70, 2065)

In [90]:
model=Sequential()
model.add(SimpleRNN(units=20,activation='tanh'))
model.add(Dense(units=20,activation='relu'))
model.add(Dense(units=10,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,batch_size=32,epochs=10,validation_split=.2)

Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.6394 - loss: 0.6726 - val_accuracy: 0.9900 - val_loss: 0.5395
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.8125 - loss: 0.5329 - val_accuracy: 1.0000 - val_loss: 0.2014
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9150 - loss: 0.2887 - val_accuracy: 1.0000 - val_loss: 0.0360
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.9700 - loss: 0.1207 - val_accuracy: 1.0000 - val_loss: 0.0105
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.9919 - loss: 0.0559 - val_accuracy: 1.0000 - val_loss: 0.0042
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.9969 - loss: 0.0304 - val_accuracy: 1.0000 - val_loss: 0.0017
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.9994 - loss: 0.0182 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9994 - loss: 0.0120 - val_accuracy: 1.0000 - v

In [94]:
new_sample1='food was good'
new_sample2='we did not like the food'

test_corpus=[new_sample1,new_sample2]
test_corpus2=list(map(str.lower,test_corpus))
test_corpus3=list(map(clean_punch,test_corpus2))

seqs=tk.texts_to_sequences(test_corpus3)
pad_seqs=pad_sequences(seqs,padding='pre',maxlen=70,truncating='post')
X_test=to_categorical(pad_seqs,num_classes=X.shape[2])
X_test.shape

(2, 70, 2065)

In [95]:
seqs

[[11, 3, 37], [46, 155, 12, 115, 1, 11]]

In [96]:
model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


array([[0.5165712 ],
       [0.03325795]], dtype=float32)